In [1]:
!pip install xgboost==1.5.2

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd

In [3]:
X_train = pd.read_parquet('X_train.parquet')

In [4]:
X_test = pd.read_parquet('X_test.parquet')

In [5]:
y_train = X_train['label']
y_test = X_test['label']
X_train = X_train[X_train.columns.difference(['label'])]
X_test = X_test[X_test.columns.difference(['label'])]

In [7]:
import xgboost as xgb

In [8]:
xgb.__version__

'1.5.2'

In [9]:
dtrain = xgb.DMatrix(
        X_train,
        y_train )

In [10]:
dtest = xgb.DMatrix(
        X_test,
        y_test    )

In [11]:
xgb_params = {
    'max_depth':         3,
    'tree_method':       'gpu_hist',
    'objective':         'binary:logistic',
    'eval_metric':       'aucpr',
    'predictor':         'gpu_predictor',
}

In [12]:
n_rounds = 500

In [13]:
model = xgb.train(params=xgb_params, 
                       dtrain=dtrain, 
                       num_boost_round=n_rounds)

In [14]:
# aucpr baseline
y_test.sum()/len(y_test) 

0.0012154066322492814

In [15]:
y_score = model.predict(dtest)

In [17]:
from sklearn.metrics import average_precision_score

y_true = y_test.values
ap = average_precision_score(y_true, y_score)
print(f'Test Area Under Precision Recall Curve: {ap: 0.4f}')

Test Area Under Precision Recall Curve:  0.8168


In [18]:
model_path = "./xgboost_model.json"

In [19]:
model.save_model(model_path)